In [ ]:
from google.cloud import bigquery
import matplotlib.pyplot as plt
import numpy as np
# Construct a BigQuery client object.
client = bigquery.Client()

def get_areas(area_id, model, constellation):
    if constellation:
      query = f"""SELECT * FROM `oxeo-main.water.water_ts` 
                WHERE area_id = {area_id}  
                AND run_id LIKE "%-{model}-%"
                AND constellation = "{constellation}" 
                
              """
    else:
      query = f"""SELECT * FROM `oxeo-main.water.water_ts` 
                WHERE area_id = {area_id}  
                AND run_id LIKE "%-{model}-%"
                
              """
    print(query)
    query_job = client.query(query)  # Make an API request.
    return query_job.to_dataframe()

def norm(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
job = get_areas(51318547, "cnn", None)
job = job[job.run_id=="51318547-cnn-985d72fe"]
# job["area"] = norm(job.area)
job = job.sort_values(by="date").drop_duplicates(subset=["date"], keep="last", ignore_index=True)
pekel_job = get_areas(51318547, "pekel-la-alumbrera", None)
# pekel_job["area"] = norm(pekel_job.area)
pekel_job = pekel_job.sort_values(by="date").drop_duplicates(subset=["date"],keep="first")

In [ ]:
import pandas as pd
import seaborn as sns
plt.figure(figsize = (25,8))
df = pd.merge(job,pekel_job,on="date")
df["cnn_ma"] = df.area_x.rolling(100).mean()
df["pekel_ma"] = df.area_y.rolling(100).mean()
sns.lineplot(x="date", y="pekel_ma", data=df,alpha=1.0, label="Pekel", color="green", linestyle='--')
# plt.ylim(0, 1e7)
sns.lineplot(x="date", y="cnn_ma", data=df,alpha=1.0,  label="CNN", color="purple")
sns.lineplot(x="date", y="area_x", data=df,alpha=0.2, hue="constellation_x")
# sns.lineplot(x="date", y="area_y", data=df,alpha=0.3, color="red")